In [2058]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('train.csv' ,sep='|')
df

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,9634,CLINIC Operation Manager,Sumatera Utara,IDR,Manajer/Asisten Manajer,5 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Layanan Kesehatan,Praktisi/Asisten Medis",NaN,NaN,1- 50 pekerja,Makanan & Minuman/Katering/Restoran,# must have good experiences in above field# e...,14000000.0
1,26646,Management Trainee,Aceh,IDR,Supervisor/Koordinator,1 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel",Asuransi kesehatan;Bisnis (contoh: Kemeja),NaN,Lebih dari 5000 pekerja,Retail/Merchandise,Lowongan ini akan mengarahkan anda langsung ke...,NaN
2,23840,Creative Content Creator SEO,Semarang,IDR,Supervisor/Koordinator,2 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Digital Marketing",Bisnis (contoh: Kemeja);-,28 days,201 - 500 pekerja,Manufaktur/Produksi,KemampuanDapat membuat konten kreatif untuk pr...,NaN
3,15321,Supervisor Sales General Trade,Surabaya,IDR,Supervisor/Koordinator,2 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel",Tip;Asuransi kesehatan;Penglihatan;Waktu regul...,27 days,501 - 1000 pekerja,Produk Konsumen/Barang konsumen yang bergerak ...,Job DescriptionWorking with company target and...,NaN
4,13672,Compensation & Benefit (Payroll) SPV,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Kontrak,"Sumber Daya Manusia/Personalia,Sumber Daya Man...",Asuransi kesehatan;Formil (contoh: Kemeja + Da...,19 days,51 - 200 pekerja,Automobil/Mesin Tambahan Automotif/Kendaraan,JOB DESCRIPTION· Manage external client...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25391,11690,Translator Mandarin,Jawa Timur,IDR,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,NaN,Sarjana (S1),Penuh Waktu,"Lainnya,Jurnalis/Editor","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",29 days,51 - 200 pekerja,Manufaktur/Produksi,Pendidikan Minimal S1Lancar berbahasa mandarin...,7500000.0
25392,19877,Staff Admin Accounting,Jakarta Utara,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan","Asuransi kesehatan;Waktu regular, Senin - Juma...",NaN,1- 50 pekerja,Elektrikal & Elektronik,SYARAT KEAHLIAN:Melakukan data entry kas dan b...,NaN
25393,2178,Web Developer,Jakarta Selatan,IDR,Supervisor/Koordinator,2 tahun,Sarjana (S1),Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak",NaN,NaN,NaN,Manufaktur/Produksi,WEB DEVELOPER (Placement : Jakarta)The ideal c...,6000000.0
25394,11750,CFO - Fintech Sharia,Jakarta Raya,IDR,CEO/GM/Direktur/Manajer Senior,15 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan",NaN,30 days,51 - 200 pekerja,Manajemen/Konsulting HR,"The CFO will be report directly to CEO, in cha...",NaN


In [2059]:
df = df[df['salary_currency'] == 'IDR']


In [2060]:
df = df.drop(['job_description', 'salary_currency', 'job_benefits', 'id', 'job_title'], axis=1)

In [2061]:
numeric_features = ['experience_level', 'employment_type', 'career_level', 'company_process_time', 'company_size', 'education_level']
categorical_features = [ 'location', 'job_function', 'company_industry']

#### Edu Level

In [2062]:
df['education_level'].value_counts()

education_level
Sarjana (S1)                                                                                                                      8276
Sertifikat Professional, D3 (Diploma), D4 (Diploma), Sarjana (S1)                                                                 4429
Tidak terspesifikasi                                                                                                              4199
SMA, SMU/SMK/STM, Sertifikat Professional, D3 (Diploma), D4 (Diploma), Sarjana (S1)                                               2196
SMA, SMU/SMK/STM                                                                                                                  1698
Sarjana (S1), Diploma Pascasarjana, Gelar Professional, Magister (S2)                                                             1644
Sertifikat Professional, D3 (Diploma), D4 (Diploma)                                                                               1336
Sertifikat Professional, D3 (Diploma), 

In [2063]:
df['education_level'].replace({'Sarjana (S1)': 'Sarjana',
                               'Sertifikat Professional, D3 (Diploma), D4 (Diploma), Sarjana (S1)' : 'Sarjana',
                               'SMA, SMU/SMK/STM, Sertifikat Professional, D3 (Diploma), D4 (Diploma), Sarjana (S1)': 'Sarjana',
                               'Sarjana (S1), Diploma Pascasarjana, Gelar Professional, Magister (S2)' : 'Magister',
                               'SMA, SMU/SMK/STM' : 'SMA/SMK',
                               'Sertifikat Professional, D3 (Diploma), D4 (Diploma)' : 'Diploma',
                               'SMA, SMU/SMK/STM, Sarjana (S1)' : 'Sarjana',
                               'Diploma Pascasarjana, Gelar Professional, Magister (S2) ' : 'Magister',
                               'Sarjana (S1), Diploma Pascasarjana, Gelar Professional, Magister (S2), Doktor (S3)' : 'Doktor',
                               'Diploma Pascasarjana, Gelar Professional, Magister (S2), Doktor (S3)' : 'Doktor',
                               'Sertifikat Professional, D3 (Diploma), D4 (Diploma), Diploma Pascasarjana, Gelar Professional, Magister (S2)' : 'Magister',
                               'SMA, SMU/SMK/STM, Sarjana (S1), Diploma Pascasarjana, Gelar Professional, Magister (S2)' : 'Magister',
                               'Doktor (S3)' : 'Doktor',
                               'Sarjana (S1), Doktor (S3)' : 'Doktor',
                               'SMA, SMU/SMK/STM, Sertifikat Professional, D3 (Diploma), D4 (Diploma), Diploma Pascasarjana, Gelar Professional, Magister (S2)' : 'Magister',
                               'Sarjana (S1), Diploma Pascasarjana, Gelar Professional' : 'Sarjana',
                               'Sertifikat Professional, D3 (Diploma), D4 (Diploma), Sarjana (S1), Doktor (S3)' : 'Doktor',
                               'Sertifikat Professional, D3 (Diploma), D4 (Diploma), Sarjana (S1), Diploma Pascasarjana, Gelar Professional, Magister (S2)' : 'Magister',
                               'SMA, SMU/SMK/STM, Sertifikat Professional, D3 (Diploma), D4 (Diploma)' : 'Diploma',
                               'Diploma Pascasarjana, Gelar Professional, Magister (S2)' : 'Magister'}, inplace=True)

# Definisikan pemetaan tipe pekerjaan ke kode yang diinginkan
education_level_mapping = {
    'Tidak terspesifikasi ': 0,
    'SMA/SMK': 1,
    'Diploma ': 2,   
    'Sarjana': 3,
    'Magister': 4,
    'Doktor': 5,
}

df['education_level'] = df['education_level'].map(education_level_mapping)

#### Company Size

In [2064]:
df['company_size'].value_counts()

company_size
51 - 200 pekerja           5534
1- 50 pekerja              3866
201 - 500 pekerja          3257
501 - 1000 pekerja         2610
2001 - 5000 pekerja        2101
Lebih dari 5000 pekerja    2076
1001 - 2000 pekerja        1864
Name: count, dtype: int64

In [2065]:
mapping = {
    '1- 50 pekerja': 25,
    '51 - 200 pekerja': 125,
    '201 - 500 pekerja': 350,
    '501 - 1000 pekerja': 750,
    '1001 - 2000 pekerja': 1500,
    '2001 - 5000 pekerja': 3500,
    'Lebih dari 5000 pekerja': 5000
}

# Konversi kolom 'company_size' ke tipe data integer menggunakan mapping
df['company_size'] = df['company_size'].map(mapping)

# Tampilkan hasil
df['company_size'].value_counts()

company_size
125.0     5534
25.0      3866
350.0     3257
750.0     2610
3500.0    2101
5000.0    2076
1500.0    1864
Name: count, dtype: int64

#### Company Process Time

In [2066]:
df['company_process_time'].value_counts()

company_process_time
29 days    3427
28 days    2330
27 days    1364
26 days    1291
25 days     988
24 days     945
21 days     836
30 days     622
22 days     565
23 days     529
19 days     469
16 days     464
20 days     397
17 days     366
14 days     360
12 days     304
18 days     300
13 days     289
1 day       261
9 days      242
11 days     224
2 days      223
6 days      198
15 days     188
4 days      187
3 days      182
7 days      176
5 days      156
8 days      145
10 days     113
Name: count, dtype: int64

In [2067]:
df['company_process_time'] = df['company_process_time'].str.replace(' days', '')
df['company_process_time'] = df['company_process_time'].str.replace(' day', '')
df['company_process_time'] = df['company_process_time'].fillna(df['company_process_time'].mode()[0])

df['company_process_time'] = df['company_process_time'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25385 entries, 0 to 25395
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   location              25385 non-null  object 
 1   career_level          25385 non-null  object 
 2   experience_level      21986 non-null  object 
 3   education_level       19248 non-null  float64
 4   employment_type       24334 non-null  object 
 5   job_function          25385 non-null  object 
 6   company_process_time  25385 non-null  int32  
 7   company_size          21308 non-null  float64
 8   company_industry      24211 non-null  object 
 9   salary                5079 non-null   float64
dtypes: float64(3), int32(1), object(6)
memory usage: 2.0+ MB


In [2068]:
 df['company_process_time'].value_counts()

company_process_time
29    10671
28     2330
27     1364
26     1291
25      988
24      945
21      836
30      622
22      565
23      529
19      469
16      464
20      397
17      366
14      360
12      304
18      300
13      289
1       261
9       242
11      224
2       223
6       198
15      188
4       187
3       182
7       176
5       156
8       145
10      113
Name: count, dtype: int64

#### EXP Level

In [2069]:
df['experience_level'].value_counts()

experience_level
1 tahun                7882
2 tahun                6106
3 tahun                4051
5 tahun                2354
4 tahun                 695
10 tahun                293
7 tahun                 207
8 tahun                 178
6 tahun                 120
15 tahun                 51
12 tahun                 23
9 tahun                  10
20 tahun                  7
16 tahun                  3
Lebih dari 20 Tahun       2
17 tahun                  1
11 tahun                  1
13 tahun                  1
14 tahun                  1
Name: count, dtype: int64

In [2070]:
df = df[df['experience_level'] != 'Lebih dari 20 Tahun']

mapping = {
    '1 tahun': 1, '2 tahun': 2, '3 tahun': 3, '4 tahun': 4, '5 tahun': 5,
    '6 tahun': 6, '7 tahun': 7, '8 tahun': 8, '9 tahun': 9, '10 tahun': 10,
    '11 tahun': 11, '12 tahun': 12, '13 tahun': 13, '14 tahun': 14,
    '15 tahun': 15, '16 tahun': 16, '17 tahun': 17, '20 tahun': 20
}

# Konversi kolom ke tipe data integer
df['experience_level'] = df['experience_level'].map(mapping)

# Tampilkan hasil
df['experience_level'].value_counts()

experience_level
1.0     7882
2.0     6106
3.0     4051
5.0     2354
4.0      695
10.0     293
7.0      207
8.0      178
6.0      120
15.0      51
12.0      23
9.0       10
20.0       7
16.0       3
17.0       1
11.0       1
13.0       1
14.0       1
Name: count, dtype: int64

#### Employment Type

In [2071]:
df['employment_type'].value_counts()

employment_type
Penuh Waktu                 20814
Kontrak                      3045
Paruh Waktu                   200
Magang                        158
Temporer                       81
Penuh Waktu, Kontrak           30
Temporer, Magang                2
Penuh Waktu, Paruh Waktu        1
Penuh Waktu, Magang             1
Name: count, dtype: int64

In [2072]:
values_to_drop = [
    'Penuh Waktu, Kontrak',
    'Temporer, Magang',
    'Penuh Waktu, Paruh Waktu',
    'Penuh Waktu, Magang'
]
df = df[~df['employment_type'].isin(values_to_drop)]

# Mapping ordinal
mapping = {
    'Penuh Waktu': 4,
    'Kontrak': 3,
    'Paruh Waktu': 2,
    'Temporer': 1,
    'Magang': 0,
}

# Konversi kolom ke tipe data integer
df['employment_type'] = df['employment_type'].map(mapping)
df['employment_type'].value_counts()

employment_type
4.0    20814
3.0     3045
2.0      200
0.0      158
1.0       81
Name: count, dtype: int64

#### Career level

In [2073]:
df['career_level'].value_counts()

career_level
Pegawai (non-manajemen & non-supervisor)             14046
Supervisor/Koordinator                                4758
Manajer/Asisten Manajer                               3357
Lulusan baru/Pengalaman kerja kurang dari 1 tahun     1744
Tidak Terspesifikasi                                  1099
CEO/GM/Direktur/Manajer Senior                         345
Name: count, dtype: int64

In [2074]:
df = df[df['career_level'] != 'Tidak Terspesifikasi']

In [2075]:

import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=['career_level'], return_df=True,
                            mapping=[{'col':'career_level','mapping':{'Lulusan baru/Pengalaman kerja kurang dari 1 tahun': 0,
                                                                  'Pegawai (non-manajemen & non-supervisor)': 1,
                                                                  'Supervisor/Koordinator': 2,
                                                                  'Manajer/Asisten Manajer': 3,
                                                                  'CEO/GM/Direktur/Manajer Senior': 4}
}])
df_encoded = encoder.fit_transform(df)
print(df_encoded.info())
df = df_encoded
df['career_level'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 24250 entries, 0 to 25395
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   location              24250 non-null  object 
 1   career_level          24250 non-null  int32  
 2   experience_level      21954 non-null  float64
 3   education_level       19213 non-null  float64
 4   employment_type       24250 non-null  float64
 5   job_function          24250 non-null  object 
 6   company_process_time  24250 non-null  int32  
 7   company_size          20500 non-null  float64
 8   company_industry      23327 non-null  object 
 9   salary                5074 non-null   float64
dtypes: float64(5), int32(2), object(3)
memory usage: 1.9+ MB
None


career_level
1    14046
2     4758
3     3357
0     1744
4      345
Name: count, dtype: int64

In [2076]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [2077]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [2078]:
models = [
    ('xgb', XGBRegressor(random_state=42)),
    ('lr', LinearRegression()),
    ('dt', DecisionTreeRegressor()),
    # ('svr', SVR()),
    ('rf', RandomForestRegressor(random_state=42))
]


In [2079]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24250 entries, 0 to 25395
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   location              24250 non-null  object 
 1   career_level          24250 non-null  int32  
 2   experience_level      21954 non-null  float64
 3   education_level       19213 non-null  float64
 4   employment_type       24250 non-null  float64
 5   job_function          24250 non-null  object 
 6   company_process_time  24250 non-null  int32  
 7   company_size          20500 non-null  float64
 8   company_industry      23327 non-null  object 
 9   salary                5074 non-null   float64
dtypes: float64(5), int32(2), object(3)
memory usage: 1.9+ MB


In [2080]:
df_train = df.dropna(subset=['salary'])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5074 entries, 0 to 25393
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   location              5074 non-null   object 
 1   career_level          5074 non-null   int32  
 2   experience_level      4662 non-null   float64
 3   education_level       3853 non-null   float64
 4   employment_type       5074 non-null   float64
 5   job_function          5074 non-null   object 
 6   company_process_time  5074 non-null   int32  
 7   company_size          4338 non-null   float64
 8   company_industry      4918 non-null   object 
 9   salary                5074 non-null   float64
dtypes: float64(5), int32(2), object(3)
memory usage: 396.4+ KB


In [2081]:
X = df_train.drop(columns=['salary'], axis=1)

y = df_train['salary']

# Pisahkan data menjadi train set dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Voting Regressor
voting_regressor = VotingRegressor(estimators=models, n_jobs=5)

# Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', voting_regressor)])

# Training
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['experience_level',
                                                   'employment_type',
                                                   'career_level',
                                                   'education_level']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['location', 'job_function',
                                                   'compa...
                                                           max_cat_threshold=None,
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=None,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           multi_strategy=None,
                                                           n_estimators=None,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=42, ...)),
                                             ('lr', LinearRegression()),
                                             ('dt', DecisionTreeRegressor()),
                                             ('rf',
                                              RandomForestRegressor(random_state=42))],
                                 n_jobs=5))])

In [2082]:


# Evaluation
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

Mean Squared Error: 12154028705749.521
Mean Absolute Error: 1838849.692978619
R2 Score: 0.5898732100026014


In [2083]:
df_pred = pd.read_csv('cleaned2.csv')
df_pred

,id,location,career_level,experience_level,education_level,employment_type,job_function,company_process_time,company_size,company_industry
0,13973,Jakarta Raya,0,NaN,3.0,4,"Penjualan / Pemasaran,Penjualan - Korporasi",21,25.0,Manajemen/Konsulting HR
1,12830,Jakarta Barat,3,3.0,NaN,4,"Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan",25,25.0,Retail/Merchandise
2,25858,Jakarta Pusat,3,5.0,4.0,4,"Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan",19,25.0,Manajemen/Konsulting HR
3,24089,Medan,1,1.0,3.0,4,"Penjualan / Pemasaran,Penjualan Ritel",16,350.0,Perbankan/Pelayanan Keuangan
4,2140,Jakarta Utara,3,5.0,NaN,4,"Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran",17,25.0,Makanan & Minuman/Katering/Restoran
...,...,...,...,...,...,...,...,...,...,...
1266,4447,Jakarta Raya,3,3.0,4.0,4,"Sumber Daya Manusia/Personalia,Sumber Daya Man...",29,NaN,NaN
1267,7628,Jakarta Selatan,0,NaN,3.0,0,"Seni/Media/Komunikasi,Periklanan",29,NaN,Seni/Desain/Fashion
1268,10276,Papua,1,1.0,NaN,4,"Teknik,Mekanikal",29,125.0,Automobil/Mesin Tambahan Automotif/Kendaraan
1269,6490,Cikarang,3,2.0,3.0,4,"Teknik,Teknik Lingkungan",29,1500.0,Pelayanan Arsitek/Desain Interior


In [2084]:
sPred = df_pred.drop('id', axis=1)
sPred


,location,career_level,experience_level,education_level,employment_type,job_function,company_process_time,company_size,company_industry
0,Jakarta Raya,0,NaN,3.0,4,"Penjualan / Pemasaran,Penjualan - Korporasi",21,25.0,Manajemen/Konsulting HR
1,Jakarta Barat,3,3.0,NaN,4,"Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan",25,25.0,Retail/Merchandise
2,Jakarta Pusat,3,5.0,4.0,4,"Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan",19,25.0,Manajemen/Konsulting HR
3,Medan,1,1.0,3.0,4,"Penjualan / Pemasaran,Penjualan Ritel",16,350.0,Perbankan/Pelayanan Keuangan
4,Jakarta Utara,3,5.0,NaN,4,"Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran",17,25.0,Makanan & Minuman/Katering/Restoran
...,...,...,...,...,...,...,...,...,...
1266,Jakarta Raya,3,3.0,4.0,4,"Sumber Daya Manusia/Personalia,Sumber Daya Man...",29,NaN,NaN
1267,Jakarta Selatan,0,NaN,3.0,0,"Seni/Media/Komunikasi,Periklanan",29,NaN,Seni/Desain/Fashion
1268,Papua,1,1.0,NaN,4,"Teknik,Mekanikal",29,125.0,Automobil/Mesin Tambahan Automotif/Kendaraan
1269,Cikarang,3,2.0,3.0,4,"Teknik,Teknik Lingkungan",29,1500.0,Pelayanan Arsitek/Desain Interior


In [2085]:
hasil = pipeline.predict(sPred)

In [2086]:
hasil_series = pd.Series(hasil, name='salary')

# Gabungkan 'id' dengan hasil prediksi menjadi dataframe
result_df = pd.concat([pd.DataFrame(df_pred['id']), hasil_series], axis=1)

# Simpan dataframe ke dalam file CSV
result_df.to_csv('sub3_5.csv', index=False)

print("File sub2.csv telah disimpan.")

File sub2.csv telah disimpan.
